# Project Title
### Data Engineering Capstone Project

#### Project Summary
The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
!pip install datapackage
!pip install geopy
!pip install pycountry-convert
!pip install s3fs
!pip install boto
import pandas as pd
import boto
import s3fs
from datapackage import Package
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import requests
from pyspark.sql.functions import isnan, when, count, col
import os
import configparser
from pyspark.sql.functions import lit 
import pyspark.sql.types as t
from datetime import datetime
from datetime import timedelta
import boto3
from io import StringIO
import sys
config = configparser.ConfigParser()
config.read('dl.cfg')
os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

    100% |████████████████████████████████| 92kB 5.7MB/s ta 0:00:011
    100% |████████████████████████████████| 71kB 8.9MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 7.3MB/s ta 0:00:011
    100% |████████████████████████████████| 51kB 8.4MB/s ta 0:00:011
    100% |████████████████████████████████| 133kB 7.8MB/s ta 0:00:01
    100% |████████████████████████████████| 245kB 8.1MB/s ta 0:00:01
  Running setup.py bdist_wheel for unicodecsv ... done
  Stored in directory: /root/.cache/pip/wheels/a6/09/e9/e800279c98a0a8c94543f3de6c8a562f60e51363ed26e71283
  Running setup.py bdist_wheel for linear-tsv ... done
  Stored in directory: /root/.cache/pip/wheels/3f/8a/cb/38917fd1ef4356b9870ace7331b83417dc594bf2c029bd991f
Successfully built unicodecsv linear-tsv
  Found existing installation: openpyxl 2.5.0b1
    Uninstalling openpyxl-2.5.0b1:
      Successfully uninstalled openpyxl-2.5.0b1
    100% |████████████████████████████████| 112kB 4.5MB/s ta 0:00:01
    100% |███████████

/opt/conda/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Step 1

### Scope
The scope of this porject is to build a datalake with some demographic, geographic data (temperature, us-cities demograpgics and imigration data) and data about airports and it's cities. Once the data is stored and transformed in a refined form, this could result in a statiscal model like a Logistic Regression, to understand for example, what are the demograpgics characteristics of the US Cities that attract people from around the world. We could also use a time series model like The SARIMA model class, to forecast the temperature and understand climtate's change.
### Describe and Gather Data
There are 4 datasets:
- airport-codes: has some informations from airports arround the round, like it's cordinates, the country, city, etc. The data was provided by Datahub.
- Imigration data: this data come's from the US National Tourism and Trade Office. Here there are data about international visitor arrival statistics by world regions and select countries (including top 20), type of visa, mode of transportation, age groups, states visited (first intended address only).
- us-cities-demographics: this data was provided by OpenSoft, and contains demographics information about the cities of the USA,  like median age, born average, male and female population, etc.
-  world temperature data: this data is provided by Kaggle, and has data about the temperature arround the world, for the purpose of studying the climate change.The data is a time series of many locations arround the world, containing the average temperature and average temperature uncertainty

In [37]:
def collect_raw_data():
    '''
    get the data with the API
    response=requests.get('https://public.opendatasoft.com/api/records/1.0/download/?dataset=us-cities-demographics')

    raw_df = pd.DataFrame([x.split(';') for x in response.text.split('\n')],)
    headers = raw_df.iloc[0]
    df_us_cities  = pd.DataFrame(raw_df.values[1:], columns=headers)
    '''
    df_us_cities=pd.read_csv('us-cities-demographics.csv',sep=';')
    df_us_cities.head(1)

    # Airport data
    df_airport=pd.read_csv('airport-codes_csv.csv')


    # Temperature data
    fname = '../../data2/GlobalLandTemperaturesByCity.csv'
    df_temperature = pd.read_csv(fname)


    spark = SparkSession \
            .builder \
            .config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11,org.apache.hadoop:hadoop-aws:2.7.2") \
            .config("fs.s3a.access.key", os.environ['AWS_ACCESS_KEY_ID']) \
            .config("fs.s3a.secret.key", os.environ['AWS_SECRET_ACCESS_KEY']) \
            .getOrCreate()
    df_imigration =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat')
    return(df_us_cities,df_airport,df_temperature,df_imigration)

In [38]:
df_us_cities,df_airport,df_temperature,df_imigration=collect_raw_data()

### Step 2: Explore and Assess the Data
#### Explore the Data  and Cleaning Steps
Here it's gonna be identified and document the steps of cleaning data for each dataset. The function check_dataframe was build to check the datatype and the NaN values for each column
##### check_dataframe function:

In [25]:
def check_dataframe(df,spark):
    if spark==False:
        df_size=len(df)
        df_duplicated=len(df.drop_duplicates())
        df_any_na=len(df[df.isna().any(axis=1)])
        if (df_size == df_duplicated):
            print ('No Duplicated data. Data has',df_size,'rows.')
            print('There are', df_any_na,'rows with data missing in at least one column. This represents',round((df_any_na/df_size)*100,2),'% of rows.')
        else:
            print('Duplicated data, needs check')
            print('There are', df_any_na,'rows with data missing in at least one column. This represents',round((df_any_na/df_size)*100,2),'% of rows.')
        print('Counts of NaN in columns')
        df_na=pd.DataFrame(df.isna().sum().rename('NaN count'))
        df_types=pd.DataFrame(df.dtypes.rename('types'))
        df_summary=df_na.merge(df_types,left_index=True, right_index=True)
        return (df_summary)
    
    else:
        df_size=df.count()
        df_duplicated=(df_size - df.drop_duplicates().count())
        if df_duplicated==0:
            print ('No Duplicated data. Data has',df_size,'rows.')
        else:
            print('Duplicated data, needs check')
        df_na=df_imigration.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df_imigration.columns]).toPandas().T
        df_types=pd.DataFrame(df_imigration.dtypes).set_index([0])
        df_summary=df_na.merge(df_types,left_index=True, right_index=True)
        df_types=pd.DataFrame(df_imigration.dtypes).set_index([0])
        df_summary=df_na.merge(df_types,left_index=True, right_index=True)
        df_summary.columns=['NaN count','types']
        return(df_summary)

## Airport data

In [26]:
df_airport.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [27]:
check_dataframe(df_airport,spark=False)

No Duplicated data. Data has 55075 rows.
There are 54397 rows with data missing in at least one column. This represents 98.77 % of rows.
Counts of NaN in columns


,NaN count,types
ident,0,object
type,0,object
name,0,object
elevation_ft,7006,float64
continent,27719,object
iso_country,247,object
iso_region,0,object
municipality,5676,object
gps_code,14045,object
iata_code,45886,object


There are some columns that have NaN values. Most of these data  could be obtained using the coordinates , since they are never missing, resulting in a good or perfect aproximation. We are going to use pycountry to do this for us.

In [28]:
# Get country code that are missing
import pycountry_convert as pc
def get_continent_name(row):
    try:
        return  (pc.country_alpha2_to_continent_code(row['iso_country']))
    except:
        return float('nan')
df_airport['continent_formated']=df_airport.apply(get_continent_name,axis=1)

# US cities data

In [29]:
df_us_cities.head(5)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [30]:
check_dataframe(df_us_cities,spark=False)

No Duplicated data. Data has 2891 rows.
There are 16 rows with data missing in at least one column. This represents 0.55 % of rows.
Counts of NaN in columns


,NaN count,types
City,0,object
State,0,object
Median Age,0,float64
Male Population,3,float64
Female Population,3,float64
Total Population,0,int64
Number of Veterans,13,float64
Foreign-born,13,float64
Average Household Size,16,float64
State Code,0,object


Despite the NaN, they repsent's just a little of the dataframe, and this rows still have a lot of data, so it doesn't make sense drop it. But we can't get this data either, so we should let this as NaN

# Temperature Data


In [31]:
df_temperature.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [32]:
check_dataframe(df_temperature,spark=False)

No Duplicated data. Data has 8599212 rows.
There are 364130 rows with data missing in at least one column. This represents 4.23 % of rows.
Counts of NaN in columns


,NaN count,types
dt,0,object
AverageTemperature,364130,float64
AverageTemperatureUncertainty,364130,float64
City,0,object
Country,0,object
Latitude,0,object
Longitude,0,object


All the data that is missing are temperature data, that we could not obtain with something like an API. Also, using the previous temperature would not be a good aproximation, since lot's of cities have missing data on consecutive days.

# Imigration data

In [39]:
pd.set_option('display.max_columns',50)
df_imigration.limit(10).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,7.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,20.0,3.0,1.0,None,None,None,T,None,None,None,1996.0,D/S,M,None,LH,346608285.0,424,F1
1,8.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,20.0,3.0,1.0,None,None,None,T,None,None,None,1996.0,D/S,M,None,LH,346627585.0,424,F1
2,9.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20480.0,17.0,2.0,1.0,None,None,None,T,N,None,M,1999.0,07152016,F,None,AF,381092385.0,338,B2
3,10.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,45.0,2.0,1.0,None,None,None,T,N,None,M,1971.0,07152016,F,None,AF,381087885.0,338,B2
4,11.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,12.0,2.0,1.0,None,None,None,T,N,None,M,2004.0,07152016,M,None,AF,381078685.0,338,B2
5,12.0,2016.0,1.0,101.0,101.0,BOS,20474.0,1.0,MA,NaN,33.0,2.0,1.0,None,None,None,T,None,None,None,1983.0,07202016,M,None,LH,406155985.0,424,B2
6,15.0,2016.0,1.0,101.0,101.0,BOS,20477.0,1.0,MA,20524.0,28.0,3.0,1.0,None,None,None,T,O,None,M,1988.0,D/S,F,None,LH,417363085.0,424,F1
7,17.0,2016.0,1.0,101.0,101.0,BOS,20480.0,1.0,MA,NaN,78.0,2.0,1.0,None,None,None,T,None,None,None,1938.0,07262016,M,None,TK,428558285.0,81,B2
8,18.0,2016.0,1.0,101.0,101.0,BOS,20480.0,1.0,MA,NaN,70.0,2.0,1.0,None,None,None,T,None,None,None,1946.0,07262016,F,None,TK,428561085.0,81,B2
9,20.0,2016.0,1.0,101.0,101.0,CHI,20473.0,1.0,IL,20482.0,28.0,2.0,1.0,None,None,None,T,O,None,M,1988.0,07192016,M,None,BA,401779785.0,295,B2


Looks like we have some columns in SAS format that we are gonna need to convert.Also, since we are only interested in the columns for building a statistical model for understanding the reasons of immirgration across the USA, we are gonna select those columns:
1. i94addr
2. biryear
3. i94port
4. arrdate
5. gender

In [40]:
pd.set_option('display.max_columns',50)
df_imigration.select('i94addr','biryear','i94port','arrdate','gender').limit(10).toPandas()

,i94addr,biryear,i94port,arrdate,gender
0,MA,1996.0,BOS,20465.0,M
1,MA,1996.0,BOS,20465.0,M
2,CT,1999.0,BOS,20469.0,F
3,CT,1971.0,BOS,20469.0,F
4,CT,2004.0,BOS,20469.0,M
5,MA,1983.0,BOS,20474.0,M
6,MA,1988.0,BOS,20477.0,F
7,MA,1938.0,BOS,20480.0,M
8,MA,1946.0,BOS,20480.0,F
9,IL,1988.0,CHI,20473.0,M


In [41]:
def date_add_(date, days):

    # Type check and convert to datetime object
    # Format and other things should be handle more delicately
    if type(date) is not datetime:
        date = datetime.strptime('1960-01-01', "%Y-%m-%d")
    return date + timedelta(days)


date_add_udf = F.udf(date_add_, t.DateType())
df_imigration = df_imigration.withColumn('sas_date', lit("1960-01-01"))
df_imigration=df_imigration.withColumn('actual_arrival_date', date_add_udf(F.to_date('sas_date'), 'arrdate')).drop('sas_date')
df_imigration.limit(10).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,actual_arrival_date
0,7.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,20.0,3.0,1.0,None,None,None,T,None,None,None,1996.0,D/S,M,None,LH,346608285.0,424,F1,2016-01-12
1,8.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,20.0,3.0,1.0,None,None,None,T,None,None,None,1996.0,D/S,M,None,LH,346627585.0,424,F1,2016-01-12
2,9.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20480.0,17.0,2.0,1.0,None,None,None,T,N,None,M,1999.0,07152016,F,None,AF,381092385.0,338,B2,2016-01-16
3,10.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,45.0,2.0,1.0,None,None,None,T,N,None,M,1971.0,07152016,F,None,AF,381087885.0,338,B2,2016-01-16
4,11.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,12.0,2.0,1.0,None,None,None,T,N,None,M,2004.0,07152016,M,None,AF,381078685.0,338,B2,2016-01-16
5,12.0,2016.0,1.0,101.0,101.0,BOS,20474.0,1.0,MA,NaN,33.0,2.0,1.0,None,None,None,T,None,None,None,1983.0,07202016,M,None,LH,406155985.0,424,B2,2016-01-21
6,15.0,2016.0,1.0,101.0,101.0,BOS,20477.0,1.0,MA,20524.0,28.0,3.0,1.0,None,None,None,T,O,None,M,1988.0,D/S,F,None,LH,417363085.0,424,F1,2016-01-24
7,17.0,2016.0,1.0,101.0,101.0,BOS,20480.0,1.0,MA,NaN,78.0,2.0,1.0,None,None,None,T,None,None,None,1938.0,07262016,M,None,TK,428558285.0,81,B2,2016-01-27
8,18.0,2016.0,1.0,101.0,101.0,BOS,20480.0,1.0,MA,NaN,70.0,2.0,1.0,None,None,None,T,None,None,None,1946.0,07262016,F,None,TK,428561085.0,81,B2,2016-01-27
9,20.0,2016.0,1.0,101.0,101.0,CHI,20473.0,1.0,IL,20482.0,28.0,2.0,1.0,None,None,None,T,O,None,M,1988.0,07192016,M,None,BA,401779785.0,295,B2,2016-01-20


In [42]:
check_dataframe(df_imigration,spark=True)

No Duplicated data. Data has 2847924 rows.


,NaN count,types
0,,
cicid,0,double
i94yr,0,double
i94mon,0,double
i94cit,0,double
i94res,0,double
i94port,0,string
arrdate,0,double
i94mode,60,double
i94addr,177129,string


## Step 3: The data model
The conceptual model that is gonna be used is a datalake. Since the purpose of the project is to collect, store and transform data that a team of data science could work, resulting for example a statiscal model / machine learning model, we need the data stored in the raw form, and in the refined form. For this, we are gonna use the AWS S3. Basicly, these are the steps necessaries:
1. Collect the data.
2. Load the data in it's raw form, that we are gonna call "transient" in S3.
3. Refine the data dealing with NaN, duplicates.
4. Load the data refined back to S3  and loading in the .parquet partioned if the data is too large.
5. Run some data quality checks.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [47]:
def collect_raw_data():
    '''
    get the data with the API
    response=requests.get('https://public.opendatasoft.com/api/records/1.0/download/?dataset=us-cities-demographics')

    raw_df = pd.DataFrame([x.split(';') for x in response.text.split('\n')],)
    headers = raw_df.iloc[0]
    df_us_cities  = pd.DataFrame(raw_df.values[1:], columns=headers)
    '''
    df_us_cities=pd.read_csv('us-cities-demographics.csv',sep=';')
    df_us_cities.head(1)

    # Airport data
    df_airport=pd.read_csv('airport-codes_csv.csv')


    # Temperature data
    fname = '../../data2/GlobalLandTemperaturesByCity.csv'
    df_temperature = pd.read_csv(fname)
    # Creating spark session
    spark = SparkSession \
            .builder \
            .config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11,org.apache.hadoop:hadoop-aws:2.7.2") \
            .config("fs.s3a.access.key", os.environ['AWS_ACCESS_KEY_ID']) \
            .config("fs.s3a.secret.key", os.environ['AWS_SECRET_ACCESS_KEY']) \
            .getOrCreate()
    df_imigration =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat')
    return(df_us_cities,df_airport,df_temperature,df_imigration)

In [48]:
def load_to_s3_transient(dataframe,folder_name,s3_filename,spark):
    if spark==False:
        bytes_to_write = dataframe.to_csv(None).encode()
        fs = s3fs.S3FileSystem(key=os.environ['AWS_ACCESS_KEY_ID'], secret=os.environ['AWS_SECRET_ACCESS_KEY'])
        with fs.open('s3://myawsbuckethermit97/transient/{0}/{1}.csv'.format(folder_name,s3_filename), 'wb') as f:
            f.write(bytes_to_write)
    else:
        dataframe.write.mode('overwrite').format("csv").save('s3a://myawsbuckethermit97/transient/{0}/{1}.csv'.format(folder_name,s3_filename))        

In [49]:
def tansform_data(df_airport,df_imigration):
    import pyspark.sql.functions as f
    import pyspark.sql.types as t
    from datetime import datetime
    from datetime import timedelta
        # Getting continent for airport data
    def get_continent_name(row):
        try:
            return  (pc.country_alpha2_to_continent_code(row['iso_country']))
        except:
            return float('nan')
        df_airport['iso_country']=df_airport.apply(get_continent_name,axis=1)
    
    # Selecting columns in imigration data
    df_imigration=df_imigration.select('i94addr','biryear','i94port','arrdate','gender','airline')
    
    # Formating sas date
    def date_add_(date, days):
        if type(date) is not datetime:
            date = datetime.strptime('1960-01-01', "%Y-%m-%d")
        return date + timedelta(days)
    date_add_udf = f.udf(date_add_, t.DateType())
    df_imigration = df_imigration.withColumn('sas_date', lit("1960-01-01"))
    df_imigration=df_imigration.withColumn('actual_arrival_date', date_add_udf(f.to_date('sas_date'), 'arrdate')).drop('sas_date')
    return(df_airport,df_imigration)

In [50]:
def load_to_s3_refined(dataframe,folder_name,s3_filename,spark):
    if spark==False:
        bytes_to_write = dataframe.to_csv(None).encode()
        fs = s3fs.S3FileSystem(key=os.environ['AWS_ACCESS_KEY_ID'], secret=os.environ['AWS_SECRET_ACCESS_KEY'])
        with fs.open('s3://myawsbuckethermit97/refined/{0}/{1}.csv'.format(folder_name,s3_filename), 'wb') as f:
            f.write(bytes_to_write)
    else:
        dataframe.write.mode('overwrite').format("csv").save('s3a://myawsbuckethermit97/refined/{0}/{1}.csv'.format(folder_name,s3_filename))        

In [51]:
def etl():
    print('Collecting data')
    df_us_cities,df_airport,df_temperature,df_imigration=collect_raw_data()
    print('Loading regular raw data into S3!')
    load_to_s3_transient(df_airport,'airport_data','airport-codes',spark=False)
    load_to_s3_transient(df_temperature,'temperature_data','world_temperature_time_series',spark=False)
    load_to_s3_transient(df_us_cities,'us_cities_data','us_cities_demographics',spark=False)
    print('Loading raw big data into S3!')
    load_to_s3_transient(df_imigration,'imigration_data','sas_data',spark=True)
    print('Transforming Data!')
    df_airport,df_imigration=tansform_data(df_airport,df_imigration)
    print('Loading regular refined data into S3!')
    load_to_s3_refined(df_airport,'airport_data','airport-codes',spark=False)
    print('Loading refined big data into S3!')
    load_to_s3_refined(df_imigration,'imigration_data','sas_data',spark=True)

In [52]:
etl()

Loading regular raw data into S3!
Loading raw big data into S3!
Transforming Data!
Loading regular refined data into S3!
Loading refined big data into S3!


#### 4.2 Data Quality Checks
1. Check if files are present on S3
2. Count of rows in each csv file

In [54]:
client = boto3.client('s3', aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
        aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'])

my_bucket = 'myawsbuckethermit97'

s3_files=[]
for key in client.list_objects(Bucket=my_bucket)['Contents']:
    s3_files.append(key['Key'])

In [55]:
def check_files(state):
    if state=='transient':
        datasets=['airport','us_cities','temperature']
    elif state=='refined':
        datasets=['airport','imigration']
    
    for dataset in datasets:
        count_occurence=(sum(dataset in file for file in s3_files))
        if count_occurence==0:
            raise ValueError('{} is not in s3!'.format(dataset))
        else:
            print(dataset,'is present in the {} state!'.format(state))

In [58]:
def check_files_lenght():
    for file in s3_files:
        if 'SUCCESS' not in file:
            print(file)
            object_key = file
            csv_obj = client.get_object(Bucket=my_bucket, Key=object_key)
            body = csv_obj['Body']
            csv_string = body.read().decode('utf-8')
            df_check = pd.read_csv(StringIO(csv_string))
            file_lenght=len(df_check)
            if file_lenght>0:
                print('Csv is fine. Data has {} rows'.format(file_lenght))
            if file_lenght==0:
                raise ValueError('The csv is empty!')

In [59]:
check_files('transient')
check_files('refined')
check_files_lenght()

airport is present in the transient state!
us_cities is present in the transient state!
temperature is present in the transient state!
airport is present in the refined state!
imigration is present in the refined state!
refined/airport_data/airport-codes.csv
Csv is fine. Data has 55075 rows
refined/imigration_data/sas_data.csv/part-00000-7283a4e0-f32e-4761-a243-0626eeece1b2-c000.csv
Csv is fine. Data has 219267 rows
refined/imigration_data/sas_data.csv/part-00001-7283a4e0-f32e-4761-a243-0626eeece1b2-c000.csv
Csv is fine. Data has 220159 rows
refined/imigration_data/sas_data.csv/part-00002-7283a4e0-f32e-4761-a243-0626eeece1b2-c000.csv
Csv is fine. Data has 220159 rows
refined/imigration_data/sas_data.csv/part-00003-7283a4e0-f32e-4761-a243-0626eeece1b2-c000.csv
Csv is fine. Data has 220159 rows
refined/imigration_data/sas_data.csv/part-00004-7283a4e0-f32e-4761-a243-0626eeece1b2-c000.csv
Csv is fine. Data has 220159 rows
refined/imigration_data/sas_data.csv/part-00005-7283a4e0-f32e-4761-a

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2907: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Csv is fine. Data has 219267 rows
transient/imigration_data/sas_data.csv/part-00001-a25f511f-1d97-4b5f-8cc0-822f98e665e4-c000.csv
Csv is fine. Data has 220159 rows
transient/imigration_data/sas_data.csv/part-00002-a25f511f-1d97-4b5f-8cc0-822f98e665e4-c000.csv
Csv is fine. Data has 220159 rows
transient/imigration_data/sas_data.csv/part-00003-a25f511f-1d97-4b5f-8cc0-822f98e665e4-c000.csv
Csv is fine. Data has 220159 rows
transient/imigration_data/sas_data.csv/part-00004-a25f511f-1d97-4b5f-8cc0-822f98e665e4-c000.csv
Csv is fine. Data has 220159 rows
transient/imigration_data/sas_data.csv/part-00005-a25f511f-1d97-4b5f-8cc0-822f98e665e4-c000.csv
Csv is fine. Data has 220159 rows
transient/imigration_data/sas_data.csv/part-00006-a25f511f-1d97-4b5f-8cc0-822f98e665e4-c000.csv
Csv is fine. Data has 220159 rows
transient/imigration_data/sas_data.csv/part-00007-a25f511f-1d97-4b5f-8cc0-822f98e665e4-c000.csv
Csv is fine. Data has 220159 rows
transient/imigration_data/sas_data.csv/part-00008-a25f51

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2907: DtypeWarning: Columns (18,23) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Csv is fine. Data has 220159 rows
transient/imigration_data/sas_data.csv/part-00012-a25f511f-1d97-4b5f-8cc0-822f98e665e4-c000.csv


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2907: DtypeWarning: Columns (15,18,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Csv is fine. Data has 206895 rows
transient/temperature_data/world_temperature_time_series.csv
Csv is fine. Data has 8599212 rows
transient/us_cities_data/us_cities_demographics.csv
Csv is fine. Data has 2891 rows


#### 4.3 Data dictionary 

##### Imigration Data

1. i94addr - string - the state that receives the immigrant
2. biryear - date - birth year of the immigrant
3. i94port - string - Aiport of arrival
4. arrdate - date - Date of arrival
5. gender - date - Gender of the immigrant
6. airline - imigrant's airline code 

##### Temperature Data


1. dt - string - Date of temperature
2. AverageTemperature - numeric - Average temperature
3. AverageTemperatureUncertainty - Uncertainty of temperature
4. City - string - The city that has the temperature's mesuare
5. Country - string -  The country that has the temperature's mesuare
6. Latitude - string - The latitude that has the temperature's mesuare
7. Longitude - string - The longitude that has the temperature's mesuare

##### US Cities Data

1. City - string - USA's city
2. State - string - USA's state
3. Median Age - numeric - cities's population's median age
4. Male Population - numeric - number of male population
5. Female Population - numeric - number of female population
6. Total Population - int  - number of total population
7. Number of Veterans - numeric - number of veterans in that city
8. Foreign-born - numeric - number of foreign born in that city
9. Average Household Size - numeric - 
10. State Code - string - the code of the state of the city
11. Race - string - the race that is majoritary
12. Count - int - count

#### Airport Data
1. ident - string - Identify the airport
2. type - string - Type of the airport (small, medium, large)
3. name - string - Name of the Airport
4. elevation_ft - numeric - The elevation fit metric
5. continent - string - Continent of the airport
6. iso_country - string - Country code where the airport is located
7. iso_region - string - State code where the airport is located
8. municipality - string - City where the airport is located
9. gps_code - string - Gps code where the airport is located
10. iata_code - string -  Airport code, also known as an IATA location identifier, IATA station code, or simply a location identifier
11. local_code - string - Local code where the airport is located
12. coordinates - string - Coordinates where the airport is located (latitude, longitude)

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

#### Clearly state the rationale for the choice of tools and technologies for the project:
The choice of technology of the datalake was made by some reasons:
1. Store cost and storage architeture-  S3 is known by it's cheap cost of storing data. 
2. Scalability - S3 is also know for it's scalability. According to AWS  "Amazon S3 provides an optimal foundation for a data lake because of its virtually unlimited scalability. You can seamlessly and nondisruptively increase storage from gigabytes to petabytes of content, paying only for what you use". 
3. Standardized APIs - We can easily use python and pyspark framework through the Standardized APIs  to store and retrieve data. 
4. Dealing with Big Data - As data grows, an ec2 will be necessary. With S3 you can easily integrate both solutions.

#### Propose how often the data should be updated and why:
Data should be updated monthly. This because, since we are dealing with time series data, we need a substantial period to understand and build a statistical model for the climate change and imigration, since the effect of human action in the case of the climate change data, or economical and political effects in the imigration moviment need's time.

###  Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.
 
#### The data was increased by 100x.
In this case, we probably would need an ec2 instance and also use the pyspark framework to deal with all datasets, since the volume would excessed a local machine hability to deal with de data. With s3 we would only need to increse storage.

#### The data populates a dashboard that must be updated on a daily basis by 7am every day.
S3 has a conector for Powerbi, possibiliting a direct conection every day. The ETL should run on a daily basis, and once this happens, we only need to set the gateway to update de data source everyday. There is also the possibility to use a python script as a data source, retrieving the data with boto for example.

####  The database needed to be accessed by 100+ people.
Since all the data is public, we could set the S3 bucket also as public. These way, there is no need to create 100 of IAM users.